In [2]:
import pandas as pd
import re
import pymorphy2
import math
import pdb
import pickle
from wiki_ru_wordnet import WikiWordnet
wikiwordnet = WikiWordnet()
morph = pymorphy2.MorphAnalyzer()

In [33]:
class CategoryPredictor:
    def __init__(self, label='Название', category_columns=['Номинация / Задача', 'категория', 'Иновация'], df=None):
        self.category_columns=category_columns
        self.curent_category = category_columns[0]
        self.label = label
        self.word_clouds = {}
        if type(df) == type(pd.DataFrame()):
            self.df = df

    def read_df(self):
        self.df = pd.read_excel('Датасет. Задача 1.xlsx')

    def calculate_distanses(self, word, cloud):
        scores = 0
        synset1 = wikiwordnet.get_synsets(word)
        if len(synset1) > 0:
            for s1 in synset1:
                for w in cloud:
                        synset2 = wikiwordnet.get_synsets(w)
                        if len(synset2) > 0:
                            for s2 in synset2:
                                common_hypernyms = wikiwordnet.get_common_hypernyms(s1, s2)
                                top_hip = sorted(common_hypernyms, key=lambda x: x[1] + x[2])
                                if len(top_hip) > 0:
                                    top_hip = top_hip[0]
                                    score = top_hip[1] + top_hip[2]
                                    if score != 0:
                                        scores += 1/score
        return scores

    def get_hypernyms(self, words):
        new_cloud = []
        for word in words: 
            lemmas = []
            synsets = wikiwordnet.get_synsets(word)
            for synset in synsets:
                for w in synset.get_words():
                    lemmas.append(w.lemma())
                    for hypernym in wikiwordnet.get_hypernyms(synset):
                        for h in hypernym.get_words():
                            lemmas.append(h.lemma())
                    for hyponym in wikiwordnet.get_hyponyms(synset):
                        for h in hyponym.get_words():
                            lemmas.append(h.lemma())
            lemmas = list(set(lemmas))
            new_cloud.extend(lemmas)
        return new_cloud

    def text_to_words(self, text):
        text = re.sub("[,|-|(|)|.|;|:|1|2|3|4|5|6|7|8|9|0|\n]","", text)
        text = text.replace('\\', ' ')
        text = text.replace('-', ' ')
        text = text.lower()
        words = text.split(' ')
        words = map(morph.normal_forms, words)
        words = list(map(lambda x: x[0], words))
        return words
    
    # create clouds for each category
    def create_clouds(self, category):
        df_c = self.df[self.df[self.curent_category] == category]
        claud = []
        df_c['words'].apply(lambda x: claud.extend(x))
        return claud

    def clear_clouds(self, row):
        category, cloud = row['category'], row['clouds']
        new_cloud = cloud.copy()
        for w in cloud:
            other_clouds = list(self.woc[self.woc['category'] != category]['clouds'])
            for orher_cloud in other_clouds:
                if w in orher_cloud:
                    new_cloud.pop(new_cloud.index(w))
                    break
        return new_cloud


    def find_categoris(self,
                        text,
                        category=None,
                        K_cloud=0.01,
                        K_clear_cloud=3,
                        K_big_cloud=1,
                        K_dist=1,
                        K_top=3,
                        Fast=False,
                        ):
        if category != None:
            self.woc = self.word_clouds[category]
        scores = {} # catedory: score
        words = self.text_to_words(text)
        categoris = list(pd.DataFrame(self.woc['category'].value_counts()).index)
        for c in categoris:
            score = 0
            cloud = list(self.woc[self.woc['category'] == c]['clouds'])[0]
            clear_cloud = list(self.woc[self.woc['category'] == c]['clear_clouds'])[0]
            big_cloud = list(self.woc[self.woc['category'] == c]['big_clouds'])[0]
            for w in words:
                if w in cloud:
                    num = cloud.count(w)
                    score += K_cloud * num / math.log(len(cloud))
                
                if w in big_cloud:
                    num = big_cloud.count(w)
                    score += K_clear_cloud * num / math.log(len(clear_cloud))
                
                if w in clear_cloud:
                    num = clear_cloud.count(w)
                    score += K_big_cloud * num / math.log(len(big_cloud))
                if Fast == False:
                    score += self.calculate_distanses(word=w, cloud=big_cloud) * K_dist / math.log(len(big_cloud))

            scores[c] = score
        if '-' in scores.keys():
            scores['-'] *= 10
        sorted_scores = dict(sorted(scores.items(),key=lambda item:item[1])[::-1][0:K_top])
        return sorted_scores
    
    def fit(self, category_column):
        self.curent_category = category_column
        self.df['words'] = self.df[self.label].apply(self.text_to_words)
        woc = pd.DataFrame({'category': pd.DataFrame(self.df[category_column].value_counts()).index})
        self.woc = woc
        woc['clouds'] = woc['category'].apply(self.create_clouds)
        woc['clear_clouds'] = woc.apply(self.clear_clouds, axis=1)
        woc['big_clouds'] = woc['clear_clouds'].apply(self.get_hypernyms)
        self.word_clouds[category_column] = woc

    def predict(self, labels, category): # labels is dataframe
        self.woc = self.word_clouds[category]
        labels['predictions'] = labels[self.label].apply(self.find_categoris)
        return labels

    def fit_all(self):
        for category_column in self.category_columns:
            self.fit(category_column)


In [36]:
df_test = pd.read_excel('test_dataset.xlsx')

In [37]:
Predictor = CategoryPredictor(label='Название',
                              category_columns=['Номинация / Задача', 'категория', 'Иновация'])
Predictor.read_df()
Predictor.fit_all()
#Predictor.find_categoris('Платформа для содействия внедрению новых технологий', 'Номинация / Задача')
Predictor.predict(labels=df_test, category='Номинация / Задача')

,Название,predictions
0,Платформа для содействия внедрению новых техно...,"{'Open Call': 3.0485325031092487, 'Проект буду..."
1,Высокопрочные материалы на основе керамики,"{'Проект будущего': 3.4585114953359417, 'Open ..."
2,Сайт для повышения цыфровой грамотности среи п...,"{'Лидеры инноваций': 2.99833180666056, 'Open C..."
3,Система распознаваиния преступления на основе ...,{'Персонализированная рекомендательная система...
4,Платформа по контролю озеленения,"{'Open Call': 0.9259005398964224, 'Персонализи..."
5,Интернет магазин по продаже игрушек,"{'Open Call': 13.483542228050633, 'Сервис пров..."
6,Разработка и производство полностью отечествен...,"{'Проект будущего': 2.3403000595559402, 'Open ..."


In [19]:
df.head(5)

,Месяц публикации идеи,Направление,Название,Содержание идеи,Ссылка на идею,Дата публикации идеи,Рейтинг идеи,Тема идеи,Регион
4663,Май,Национальная социальная инициатива,Создание открытого социального пространства дл...,На этапе средней школы создать для детей терри...,http://ideas.roscongress.org/improject-16467/i...,19.05.2022,55.0,Как создать в школе безопасную и доверительную...,Москва
4554,Май,Национальная социальная инициатива,Достойный гражданин России.,Воспитывать в детях патриотизм и любовь к Роди...,http://ideas.roscongress.org/improject-16467/i...,19.05.2022,54.0,Как создать в школе безопасную и доверительную...,Нижегородская область
16459,Май,Предпринимательство,Создание магазинов с интересной подборкой перс...,Создание магазинов с интересной подборкой перс...,http://ideas.roscongress.org/improject-16458/i...,20.05.2022,220.0,Какие решения помогут несырьевому бизнесу стат...,Москва
2390,Апрель,Национальная социальная инициатива,"Повышение качества и уровня жизни семей, воспи...",а базе КГБУ СО «Реабилитационный центр для дет...,http://ideas.roscongress.org/improject-16467/i...,22.04.2022,81.0,Как эффективно бороться с бедностью в России?,Красноярский край
10278,Апрель,Национальная экологическая инициатива,Подготовка рабочей версии новой редакции Клима...,\nПроблема\nизменения современного глобального...,http://ideas.roscongress.org/improject-16460/i...,04.04.2022,2203.0,"Как государству, бизнесу и населению подготови...",Москва


In [18]:
test_part = 0.3

df = pd.read_excel('/Users/lebedevmatvey/Documents/Nlp_fastWin/„¥â «́­ ï ¢ë£àă§ª  ¯® ¨¤¥ï¬ (®¡­®¢«¥­® 08.08).xlsx',
 sheet_name=1)
df = df.drop(columns=['H1', 'H2', 'H3'])
df = df.dropna()
df = df.rename(columns={'Заголовок идеи': 'Название'})
df = df.sample(frac=1)

df_train = df.head(int(len(df) * (1 - test_part)))
df_test = df.tail(int(len(df) * test_part))

In [44]:
Predictor = CategoryPredictor(label='Название',
                              category_columns=['Направление'], df=df_train)
Predictor.fit_all()

/var/folders/64/nbt92b3d4_n2r41g3gx6lm340000gn/T/ipykernel_28272/1730755043.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['words'] = self.df[self.label].apply(self.text_to_words)


In [43]:
Predictor.predict(labels=df_test, category='Направление')

KeyError: 'Направление'